In [1]:
# this notebook requires nuztf installation
%load_ext autoreload
%autoreload 2

In [1]:
from nuztf.skymap_scanner import SkymapScanner

import pandas as pd
import numpy as np



from tqdm.notebook import tqdm

In [16]:
candidate_filename = "/mnt/c/Users/Public/Documents/Uni/master/masterarbeit/ampel/ampel-results/ligo-healpix/test2/compareKasliwal_confirmed.ods"
candidate_filename = "/mnt/c/Users/Public/Documents/Uni/master/masterarbeit/ampel/ampel-results/ligo-healpix/test2/S190426c_ampel_kasliwal.ods"
candidate_filename = "/mnt/c/Users/Public/Documents/Uni/master/masterarbeit/ampel/ampel-results/ligo-healpix/test2/S200115j_rev_0/TransientTable.csv"


#data_df = pd.read_excel(candidate_filename)
data_df = pd.read_csv(candidate_filename, sep=";")

#print(data_df.head())

data_df = data_df.sort_values(by="Kilonovaness", ascending=False)

gw_map = data_df["Healpix map"].unique()
gw_names = [gw[:gw.find(".")] for gw in gw_map]
print(data_df.tail())

   Unnamed: 0  Ampel z  Ampel z (precision)  Ampel distance  \
0           0    0.128                 0.02         3.73565   
1           1      NaN                  NaN             NaN   
2           2      NaN                  NaN             NaN   

   Peak absmag (Possis)  Fit chisq (Possis)  Fit ndof (Possis)  Kilonovaness  \
0            -20.127887           23.794426                1.0            15   
1                   NaN                 NaN                NaN             8   
2                   NaN                 NaN                NaN             8   

                                             Rejects  \
0  ('distance_mismatch', 'absmag', 'PS1_photoz', ...   
1  ('no_ampelz_match', 'no_possis_fits', 'TNS', '...   
2  ('no_ampelz_match', 'no_possis_fits', 'GAIADR2...   

   Kilonovaness/Number of Checks  ...  CatMatch-kilonovaness  \
0                       0.625000  ...                      0   
1                       0.533333  ...                      0   
2        

In [17]:
for i in tqdm(range(0, len(gw_names)- 0)):
    #print(i)
    gw_name = gw_names[i]
    data_subset = (data_df[data_df["Healpix map"] == gw_map[i]]).sort_values(by="Kilonovaness", ascending=False)

    #print(data_subset.head(20))
    
    ztf_ids = np.char.strip(data_subset["ztf_id"].to_numpy(dtype="str"))

    #print(data_subset.head())

    print(gw_name, ztf_ids)
    prob_threshold = 0.9
    window = 3

    #ztf_ids = ["ZTF19aatubsj"]

    #print(ztf_ids)

    scanner = SkymapScanner(
        event=gw_name,
        prob_threshold=prob_threshold,
        n_days=window,
    )
    scanner.add_to_cache_by_names(ztf_ids=ztf_ids, with_history=False)

    print(scanner.cache.keys())

    #alert["kilonova_eval"]["kilonovaness"]

    for ztf_name, cand_dict in scanner.cache.items():
        if data_subset.get("Kilonovaness") is not None:
            #print(ztf_name, data_subset[data_subset["ztf_id"] == ztf_name]["Kilonovaness"].values)
            #print(ztf_name, data_subset[data_subset["ztf_id"] == ztf_name]["Kilonovaness"].values[0])
            kilonovaness = data_subset[data_subset["ztf_id"] == ztf_name]["Kilonovaness"].values[0]
        else:
            kilonovaness = None
        #print(ztf_name, data_subset[data_subset["ztf_id"] == ztf_name]["Kilonovaness"].values[0])

        if data_subset.get("Ampel z") is not None:
            z = data_subset[data_subset["ztf_id"] == ztf_name]["Ampel z"].values[0]
        else:
            z = None

        cand_dict.update({"kilonova_eval": {"kilonovaness": kilonovaness, "z": z}})

    print(scanner.cache.keys())

    scanner.create_candidate_summary()


    # plan: nimm aus data_df gw_name, 
    # nur für candidates mit dem gw_name, mache pdfs
    # evtl versuche kilonovaness in pdf zu integrieren?

  0%|          | 0/1 [00:00<?, ?it/s]

S200115j ['ZTF20aafqpum' 'ZTF20aafqvyc' 'ZTF20aafrftb']


  0%|          | 0/12582912 [00:00<?, ?it/s]

  0%|          | 0/980 [00:00<?, ?it/s]

dict_keys(['ZTF20aafqpum', 'ZTF20aafqvyc', 'ZTF20aafrftb'])
dict_keys(['ZTF20aafqpum', 'ZTF20aafqvyc', 'ZTF20aafrftb'])


  0%|          | 0/3 [00:00<?, ?it/s]

0.128
None
None
